In [2]:
import pandas as pd
import sys, os

sys.path.append("../")
from utils.logging import Logger01
from strategies.TaLib_SMACross import TaLib_SMACross
from datetime import datetime
from utils.basic import get_report_complete

sys.path.append("../libraries/backtrader")
import backtrader as bt

sys.path.append('../libraries/quantstats')
import quantstats as qs

ModuleNotFoundError: No module named 'quantstats'

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TaLib_SMACross)

datapath='../data/us/daily/aapl.csv'
df = pd.read_csv(datapath)
df['date'] = pd.to_datetime(df['date'])
# Create a Data Feed
data = bt.feeds.PandasData(dataname=df,datetime=0,open=1,high=2,low=3,close=4,volume=5)

df = pd.read_csv(datapath)
df["date"] = pd.to_datetime(df["date"])
# Create a Data Feed
data = bt.feeds.PandasData(
    dataname=df, datetime=0, open=1, high=2, low=3, close=4, volume=5
)

# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.resampledata(data, name=datapath.replace("/", "-").replace("\\", "-"))

log_path = f'../backtests/test_TaLib_SMACross_{datapath.replace("/","-").replace(chr(92),"-")}_{datetime.now().isoformat()}'

cerebro.addanalyzer(Logger01, log_path=log_path, data_df=df)

# Set our desired cash start
cerebro.broker.setcash(1000)
cerebro.broker.setcommission(commission=0.001, margin=None)

if not os.path.exists(log_path):
    os.makedirs(log_path)
writer_path = os.path.join(log_path, "writer.csv")
cerebro.addwriter(bt.WriterFile, csv=True, out=writer_path)

# Print out the starting conditions
print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())

# Run over everything
cerebro.run(exactbars=1)

# Print out the final result
print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())


Starting Portfolio Value: 1000.00
50 15
[LOG] - Funds logged
[LOG] - Orders logged
[LOG] - Trades logged


/home/fcelaya/quant/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


No analysis
Final Portfolio Value: 1085.17


In [ ]:
get_report_complete(log_path=log_path, html=True,console=True)